<a href="https://colab.research.google.com/github/jatindangi1206/dna-sampling-project/blob/main/Meals_Data_Combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import os

def identify_csv_type(df):
    """
    Identifies whether a dataframe is from meals or mealinfo based on column names.

    Returns: 'meals', 'mealinfo', or 'unknown'
    """
    columns = set(df.columns)
    column_str = ','.join(columns).lower()

    # Count how many columns start with each prefix
    meals_count = sum(1 for col in columns if col.startswith('meals_'))
    mealinfo_count = sum(1 for col in columns if col.startswith('mealinfo_'))

    print(f"   Debug: meals_ prefix count: {meals_count}, mealinfo_ prefix count: {mealinfo_count}")
    print(f"   Debug: columns found: {list(columns)[:5]}...")

    # Check for meals-specific columns (need at least 3 to confirm)
    meals_indicators = ['meals_id', 'meals_user_id', 'meals_mealinfoid']
    meals_matches = sum(1 for ind in meals_indicators if ind in columns)

    # Check for mealinfo-specific columns (need at least 3 to confirm)
    mealinfo_indicators = ['mealinfo_id', 'mealinfo_dish', 'mealinfo_outlet']
    mealinfo_matches = sum(1 for ind in mealinfo_indicators if ind in columns)

    print(f"   Debug: meals matches: {meals_matches}, mealinfo matches: {mealinfo_matches}")

    # Use multiple strategies to identify
    if meals_count >= 3 or meals_matches >= 3:
        return 'meals'
    elif mealinfo_count >= 3 or mealinfo_matches >= 3:
        return 'mealinfo'
    else:
        # If still unsure, check if 'dish' or 'outlet' exists (typical of mealinfo)
        if 'mealinfo_dish' in column_str or 'dish' in columns:
            return 'mealinfo'
        elif 'meals_user_id' in column_str or 'user_id' in columns:
            return 'meals'
        return 'unknown'


def find_and_combine_meal_data(folder1_path, folder2_path, output_csv_path='combined_meal_data.csv'):
    """
    Automatically identifies meals and mealinfo CSV files from two folders and combines them.

    Parameters:
    - folder1_path: Path to first folder (e.g., 'mealinfo' or 'meals')
    - folder2_path: Path to second folder (e.g., 'meals' or 'mealinfo')
    - output_csv_path: Path for the output combined CSV
    """

    print("=" * 60)
    print("SMART MEAL DATA COMBINER")
    print("=" * 60)

    # Find CSV files in both folders
    def find_csv_in_folder(folder_path):
        csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError(f"No CSV files found in {folder_path}")
        # Return the first CSV file found
        return os.path.join(folder_path, csv_files[0])

    print(f"\n📁 Searching in folder 1: {folder1_path}")
    csv_file1 = find_csv_in_folder(folder1_path)
    print(f"   Found: {os.path.basename(csv_file1)}")

    print(f"\n📁 Searching in folder 2: {folder2_path}")
    csv_file2 = find_csv_in_folder(folder2_path)
    print(f"   Found: {os.path.basename(csv_file2)}")

    # Read both CSV files
    print("\n📊 Reading CSV files...")
    df1 = pd.read_csv(csv_file1)
    df2 = pd.read_csv(csv_file2)

    # Identify which is which
    print("\n🔍 Identifying file types...")
    type1 = identify_csv_type(df1)
    type2 = identify_csv_type(df2)

    print(f"   {os.path.basename(csv_file1)} → {type1.upper()}")
    print(f"   {os.path.basename(csv_file2)} → {type2.upper()}")

    # Assign to correct variables
    if type1 == 'meals' and type2 == 'mealinfo':
        meals_df = df1
        mealinfo_df = df2
    elif type1 == 'mealinfo' and type2 == 'meals':
        meals_df = df2
        mealinfo_df = df1
    else:
        raise ValueError(f"Could not identify files correctly. Got types: {type1}, {type2}")

    print(f"\n✓ Meals records: {len(meals_df)}")
    print(f"✓ Mealinfo records: {len(mealinfo_df)}")

    # Show columns found
    print(f"\n📋 Meals columns: {list(meals_df.columns)}")
    print(f"📋 Mealinfo columns: {list(mealinfo_df.columns)}")

    # Rename columns
    print("\n🔄 Renaming columns...")
    meals_df = meals_df.rename(columns={
        'meals_id': 'meal_id',
        'meals_user_id': 'user_id',
        'meals_mealinfoid': 'mealinfo_id',
        'meals_time': 'time',
        'meals_seasonid': 'season_id',
        'meals_deleted': 'is_deleted'
    })

    mealinfo_df = mealinfo_df.rename(columns={
        'mealinfo_id': 'mealinfo_id',
        'mealinfo_dish': 'dish',
        'mealinfo_meal': 'meal',
        'mealinfo_helping': 'helping',
        'mealinfo_outlet': 'outlet',
        'mealinfo_createdat': 'created_at',
        'mealinfo_time': 'mealinfo_time',
        'mealinfo_image': 'image_url',
        'mealinfo_customization': 'customization',
        'mealinfo_servedas': 'served_as',
        'mealinfo_review': 'review',
        'mealinfo_rating': 'rating',
        'mealinfo_foodwasteimage': 'food_waste_image',
        'mealinfo_foodwaste': 'food_waste'
    })

    # Merge the dataframes
    print("\n🔗 Merging datasets...")
    combined_df = meals_df.merge(
        mealinfo_df,
        on='mealinfo_id',
        how='left'
    )

    # Select and reorder columns for the final output
    final_columns = [
        'meal_id',
        'user_id',
        'dish',
        'outlet',
        'time',
        'rating',
        'customization',
        'helping',
        'served_as',
        'review',
        'food_waste',
        'food_waste_image',
        'season_id',
        'image_url',
        'is_deleted'
    ]

    # Keep only the columns that exist
    available_columns = [col for col in final_columns if col in combined_df.columns]
    combined_df = combined_df[available_columns]

    # Save to CSV
    print(f"\n💾 Saving combined data to {output_csv_path}...")
    combined_df.to_csv(output_csv_path, index=False)

    print("\n" + "=" * 60)
    print("✅ SUCCESS!")
    print("=" * 60)
    print(f"✓ Combined CSV created: {output_csv_path}")
    print(f"✓ Total records: {len(combined_df)}")
    print(f"✓ Columns included: {len(available_columns)}")
    print(f"\n📊 Columns in output:")
    for col in available_columns:
        print(f"   • {col}")

    # Display sample data
    print("\n📋 First 3 rows of combined data:")
    print(combined_df.head(3).to_string())

    # Statistics
    print(f"\n📈 Quick Stats:")
    print(f"   • Unique users: {combined_df['user_id'].nunique()}")
    print(f"   • Unique dishes: {combined_df['dish'].nunique()}")
    if 'outlet' in combined_df.columns:
        print(f"   • Unique outlets: {combined_df['outlet'].nunique()}")
    if 'is_deleted' in combined_df.columns:
        print(f"   • Deleted records: {combined_df['is_deleted'].sum()}")

    return combined_df


# Main execution for Google Colab
if __name__ == "__main__":
    print("\n🚀 Starting Smart Meal Data Combiner...\n")

    # Simply specify the folder paths - no need to know the exact CSV filenames!
    folder1 = 'mealinfo'  # Folder containing mealinfo CSV
    folder2 = 'meals'     # Folder containing meals CSV

    # The script will automatically:
    # 1. Find the CSV files in each folder
    # 2. Identify which is meals and which is mealinfo
    # 3. Combine them properly
    # 4. Create the output CSV

    try:
        combined_data = find_and_combine_meal_data(folder1, folder2)

        # Download the combined file (for Colab)
        print("\n📥 Preparing download...")
        from google.colab import files
        files.download('combined_meal_data.csv')
        print("✓ Download started!")

    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        print("\nMake sure you have:")
        print("1. Created 'mealinfo' and 'meals' folders")
        print("2. Uploaded your CSV files to these folders")
        print("3. The CSV files have the correct column names")


🚀 Starting Smart Meal Data Combiner...

SMART MEAL DATA COMBINER

📁 Searching in folder 1: mealinfo
   Found: part-00000-2596f8c1-36af-4dde-a6f7-d4d92661f185-c000.csv

📁 Searching in folder 2: meals
   Found: part-00000-90bf1134-2018-45a4-bce3-49f4e0527fa7-c000.csv

📊 Reading CSV files...

🔍 Identifying file types...
   Debug: meals_ prefix count: 0, mealinfo_ prefix count: 14
   Debug: columns found: ['mealinfo_review', 'mealinfo_rating', 'mealinfo_foodwasteimage', 'mealinfo_dish', 'mealinfo_servedas']...
   Debug: meals matches: 0, mealinfo matches: 3
   Debug: meals_ prefix count: 6, mealinfo_ prefix count: 0
   Debug: columns found: ['meals_seasonid', 'meals_mealinfoid', 'meals_user_id', 'meals_id', 'meals_time']...
   Debug: meals matches: 3, mealinfo matches: 0
   part-00000-2596f8c1-36af-4dde-a6f7-d4d92661f185-c000.csv → MEALINFO
   part-00000-90bf1134-2018-45a4-bce3-49f4e0527fa7-c000.csv → MEALS

✓ Meals records: 104
✓ Mealinfo records: 3194

📋 Meals columns: ['meals_user_id',

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Download started!
